# Student Mental Health
## Data Preprocessing
Author: Harman Gill




## Install Libraries

In [ ]:
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 17 kB/s 


In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 3.3 MB/s 
     |████████████████████████████████| 3.1 MB 7.6 MB/s 
     |████████████████████████████████| 5.6 MB 29.6 MB/s 
     |████████████████████████████████| 142 kB 50.9 MB/s 
     |████████████████████████████████| 4.1 MB 39.2 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


## Import Libraries

In [ ]:
# Dependencies
import pandas as pd
from google.colab import files
import numpy as np

# Dependencies for Pre-processing
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import re
import pickle
import emot
from emot.emo_unicode import UNICODE_EMOJI
import gensim
from gensim import corpora
import pdfplumber

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Upload Data




In [ ]:
uploaded = files.upload()

Saving RedditPosts.csv to RedditPosts.csv


## Load the Data

In [ ]:
df = pd.read_csv('RedditPosts.csv')
preprocessed_df = df[['id','title','selftext','author','author_flair_text','created_utc','num_comments']]
preprocessed_df['post'] = preprocessed_df['title'].fillna('') + str(". ") + preprocessed_df['selftext'].fillna('')
preprocessed_df = preprocessed_df.drop(['title','selftext'], axis=1)
preprocessed_df['post'].replace('', np.nan, inplace=True)
preprocessed_df['post'].isna().sum()
preprocessed_df= preprocessed_df.dropna(subset=['post'])
print(preprocessed_df)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,9,10,25,44,68,70,76,77,78,79,81,82,83,84,85,86,87,88,89,91,93,95) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


           id                author  \
0      abcz5u           lit12345yxz   
1      abd016        PolysciStudent   
2      abdm2o       doyouevenschool   
3      abev8w         ilovedogs1017   
4      abewo8              delciopp   
...       ...                   ...   
56256  vodkit             vivian352   
56257  vodxga           WildWiggety   
56258  voh3lj             Kermolion   
56259  vojc2e                nurkee   
56260  voky74  Mysterious-Chart-930   

                                    author_flair_text  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                        MEC E Year 2   
...                                               ...   
56256                                             NaN   
56257    Prospective Student - Faculty of Engineering   
56258         Unde

In [ ]:
posts = pd.DataFrame(preprocessed_df.post)
posts.head()

,post
0,PharmD Application. I was hoping anything that...
1,Happy New Years. I wish you all a happy new ye...
2,New Year’s Resolution.... Learn to accept that...
3,Winter Term Midterms. Does anyone know if midt...
4,What's your New Years resolution?.


## Clean the Data

In [ ]:
posts['clean_posts'] = [clean(i) for i in (posts.loc[:,'post'])]

In [ ]:
print(posts)

                                                    post  \
0      PharmD Application. I was hoping anything that...   
1      Happy New Years. I wish you all a happy new ye...   
2      New Year’s Resolution.... Learn to accept that...   
3      Winter Term Midterms. Does anyone know if midt...   
4                    What's your New Years resolution?.    
...                                                  ...   
56256  Are starbucks, tim hortons...etc always hiring...   
56257  Laptop/Computer for Engineer. I am seeking rec...   
56258  Computer Game Development Certificate Question...   
56259  Unofficial Second Year ENG GPA Submission. Fir...   
56260  Student Health Care. Can anyone explain the pr...   

                                             clean_posts  
0      pharmd application hop anything applied pharmd...  
1      happy new year wish happy new year learn succe...  
2      new year ’ resolution learn accept polyscistud...  
3      winter term midterm anyone know midt

In [ ]:
def clean(text):
  _steps = [
           lowercase,
           remove_stopwords,
           remove_numbers,
           remove_line_breaks,
           remove_URL,
           remove_punctuation,
           convert_emojis,
           lemma,  
  ]
  for step in _steps:
      text = step(text)
  return(text)

### Lowercase

In [ ]:
def lowercase(text):
  low_text = [token.lower() for token in word_tokenize(text)]
  return ' '.join(low_text)

### Removing Line Breaks

In [ ]:
def remove_line_breaks(doc):
  doc = doc.replace('\r',' ').replace('\n',' ')
  return doc

### Remove URL

In [ ]:
def remove_URL(text):
    return re.sub(r"http\S+", "", text)

### Remove Numbers

In [ ]:
def remove_numbers(text):
  text = ''.join(c for c in text if not c.isdigit())
  return text

### Remove Punctuation

In [ ]:
def remove_punctuation(text):
    re_replacements = re.compile("__[A-Z]+__")
    re_punctuation = re.compile("[%s]" % re.escape(string.punctuation))
    '''Escape all the characters in pattern except ASCII letters and numbers'''
    tokens = word_tokenize(text)
    tokens_zero_punctuation = []
    for token in tokens:
        if not re_replacements.match(token):
            token = re_punctuation.sub(" ", token)
        tokens_zero_punctuation.append(token)
    return ' '.join(tokens_zero_punctuation)

### Remove Stopwords

In [ ]:
def remove_stopwords(text):
  new_stopwords = ['&amp;#x200B', 'amp', 'x200b', 'www', 'https', 'http', 'com']
  stop = nltk.corpus.stopwords.words('english')
  stop.extend(new_stopwords)
  word_tokens = nltk.word_tokenize(text)
  text = " ".join([word for word in word_tokens if word not in stop])
  return text

### Handle emojis

In [ ]:
def extract_emojis(s):
    return ''.join((' '+c+' ') if c in emot.UNICODE_EMOJI else c for c in s)

In [ ]:
def convert_emojis(text):
  text = extract_emojis(text)
  for emot in UNICODE_EMOJI:
    text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":","").split()))
  return text

### Lemmatization

In [ ]:
def lemma(text):
  lemmatizer = WordNetLemmatizer()
  word_tokens = nltk.pos_tag(nltk.word_tokenize(text))
  word_tagged = list(map(lambda x: (x[0], tagger(x[1])), word_tokens))
  lemma_text = []
  for word, tag in word_tagged:
      if tag is None:
          lemma_text.append(word)
      else:       
          lemma_text.append(lemmatizer.lemmatize(word, tag))
  lemma_text = " ".join(lemma_text)
  return lemma_text

In [ ]:
def tagger(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return None

## Form Bigrams

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([post.split() for post in (posts.loc[:,'clean_posts'])])
finder.apply_freq_filter(50)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

In [ ]:
bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis=0, ascending=False, inplace=True)

### Filter for bigrams with only noun-type structures

In [ ]:
def bigram_filter(bigram):
  tag = nltk.pos_tag(bigram)
  if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
    return False
  return True

In [ ]:
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 5, axis = 1)][:100]

bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 2 or len(x[1]) > 2]

In [ ]:
bigrams[:100]

['poid returnto',
 '\u200d female_sign',
 'kyle nash',
 'catoid poid',
 'ufile io',
 'sai yiu',
 'usp sf',
 'lisa smithson',
 'php catoid',
 'linear algebra',
 'van vliet',
 'immunology infection',
 'claire scavuzzo',
 'gordon lee',
 'female_sign ️',
 'jason lang',
 'viewform usp',
 'andrew wong',
 'red_heart ️',
 'formula sheet',
 'vice president',
 'dental hygiene',
 'index html',
 'y all',
 'auto webp',
 'pjpg auto',
 'the gateway',
 'pro con',
 'jpg width',
 'redd it',
 'hub mall',
 'preview redd',
 'format pjpg',
 'width format',
 'discord gg',
 'arc card',
 'cr ncr',
 'docs google',
 'pre reqs',
 'comp sci',
 'opt out',
 'conditional acceptance',
 'bear track',
 'pre req',
 'doc google',
 'unpopular opinion',
 'pre requisite',
 'wear mask',
 'png width',
 'sf link',
 'png auto',
 'budget cut',
 'form gle',
 'gift card',
 'civ e',
 'loudly_crying_face loudly_crying_face',
 'mec e',
 'access database',
 'cameron library',
 'statement result',
 'academic standing',
 'view poll',
 'm

### Concatenate n-grams

In [ ]:
def replace_ngram(x):
  for gram in bigrams:
    x = x.replace(gram, '_'.join(gram.split()))
  return x

In [ ]:
posts_w_ngrams = posts.copy()

In [ ]:
posts_w_ngrams.clean_posts = posts_w_ngrams.clean_posts.map(lambda x: replace_ngram(x))

In [ ]:
print(posts_w_ngrams)

                                                    post  \
0      PharmD Application. I was hoping anything that...   
1      Happy New Years. I wish you all a happy new ye...   
2      New Year’s Resolution.... Learn to accept that...   
3      Winter Term Midterms. Does anyone know if midt...   
4                    What's your New Years resolution?.    
...                                                  ...   
56256  Are starbucks, tim hortons...etc always hiring...   
56257  Laptop/Computer for Engineer. I am seeking rec...   
56258  Computer Game Development Certificate Question...   
56259  Unofficial Second Year ENG GPA Submission. Fir...   
56260  Student Health Care. Can anyone explain the pr...   

                                             clean_posts  
0      pharmd application hop anything applied pharmd...  
1      happy new year wish happy new year learn succe...  
2      new year ’ resolution learn accept polyscistud...  
3      winter term midterm anyone know midt

## Tokenize posts and remove words with less than 2 characters

In [ ]:
posts_w_ngrams = posts_w_ngrams.clean_posts.map(lambda x: [word for word in x.split() if len(word) > 2])

In [ ]:
print(posts_w_ngrams)

0        [pharmd, application, hop, anything, applied, ...
1        [happy, new, year, wish, happy, new, year, lea...
2        [new, year, resolution, learn, accept, polysci...
3        [winter, term, midterm, anyone, know, midterm,...
4                                  [new, year, resolution]
                               ...                        
56256    [starbucks, tim, hortons, etc, always, hire, i...
56257    [laptop, computer, engineer, seek, recommendat...
56258    [computer, game, development, certificate, que...
56259    [unofficial, second, year, eng, gpa, submissio...
56260    [student, health, care, anyone, explain, proce...
Name: clean_posts, Length: 56261, dtype: object


## Add the preprocessed posts to dataframe

In [ ]:
preprocessed_df['preprocessed_post'] = posts_w_ngrams.values

In [ ]:
preprocessed_df = preprocessed_df.reset_index(drop=True)

In [ ]:
print(preprocessed_df)

           id                author  \
0      abcz5u           lit12345yxz   
1      abd016        PolysciStudent   
2      abdm2o       doyouevenschool   
3      abev8w         ilovedogs1017   
4      abewo8              delciopp   
...       ...                   ...   
56256  vodkit             vivian352   
56257  vodxga           WildWiggety   
56258  voh3lj             Kermolion   
56259  vojc2e                nurkee   
56260  voky74  Mysterious-Chart-930   

                                    author_flair_text  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                        MEC E Year 2   
...                                               ...   
56256                                             NaN   
56257    Prospective Student - Faculty of Engineering   
56258         Unde

In [ ]:
preprocessed_df.to_csv('PreprocessedPosts.csv', header=True, index=False)

In [ ]:
preprocessed_df.to_json('PreprocessedPosts.json', orient='columns')